In [1]:
# Load the TensorBoard notebook extension.
%load_ext tensorboard

from datetime import datetime

import tensorflow as tf
from tensorflow import keras
import tensorflow.keras.layers as Layer

import tensorboard
import matplotlib.pyplot as plt

import numpy as np

In [2]:
# load the dataset and normalize
(train_images, train_labels), (test_images, test_labels) = keras.datasets.cifar10.load_data()
train_images = train_images / 255.0
test_images = test_images / 255.0

x_train = train_images.reshape(-1, 32, 32, 3) #add an additional dimension to represent the single-channel
x_test = test_images.reshape(-1, 32, 32, 3)

In [ ]:
# visualize data by plotting images
fig, ax = plt.subplots(5, 5)
k = 0
 
for i in range(5):
    for j in range(5):
        ax[i][j].imshow(x_train[k], aspect='auto')
        k += 1
 
plt.show()

# Convolutional LeNet Model

In [ ]:
le_net_model = tf.keras.models.Sequential(name='CNN_LeNet')

le_net_model.add(Layer.Conv2D(6, (5, 5), padding='same', activation='relu'))
le_net_model.add(Layer.MaxPooling2D(pool_size=(2, 2), strides=(2,2)))

le_net_model.add(Layer.Conv2D(16, (5, 5), padding='same', activation='relu'))
le_net_model.add(Layer.MaxPooling2D(pool_size=(2, 2), strides=(2,2)))

le_net_model.add(Layer.Conv2D(120, (5, 5), padding='same', activation='relu'))

le_net_model.add(Layer.Flatten())
le_net_model.add(Layer.Dense(84))
le_net_model.add(Layer.Activation('relu'))

le_net_model.add(Layer.Dense(10))
le_net_model.add(Layer.Activation('softmax'))

le_net_model.compile(loss='sparse_categorical_crossentropy', optimizer=tf.keras.optimizers.Adam(learning_rate=.001), metrics=['accuracy'])

le_net_model.build(input_shape=(1,32,32,3))

le_net_model.summary()

# Define the Keras TensorBoard callback.
logdir="logs/fit/" + datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = keras.callbacks.TensorBoard(log_dir=logdir)
early_stopping_callback = tf.keras.callbacks.EarlyStopping(
    monitor='accuracy', min_delta=.0001, patience=3, verbose=5,
    mode='auto', baseline=None, restore_best_weights=False
)



# Train the model.
le_net_model.fit(
    x_train,
    train_labels, 
    batch_size=512,
    epochs=25,
    callbacks=[tensorboard_callback, early_stopping_callback])

# Evaluate
score = le_net_model.evaluate(x_test, test_labels)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

# Report For Convolutional LeNet Model
<br>

## Question 1

The learning rate, though important, didn't seem to have as big as an effect when using the Adam optimizer. For example, the validation accuracies for .001, .0001, and .01 learning rates were respectively .963, .961, and .958. This shows that there was only a .005 accuracy difference from the lowest to the highest accuracies. This indicates that at least when using the Adam optimizer on this specific problem, the learning rate doesn't have a huge impact on the validation accuracy. The best overall learning rate proved to be .001.


## Question 2

The batch size does have a bigger impact on the validation accuracy than the learning rate did. I tested 4 different batch sizes ([64, 128, 512, 1024]) and there was a much larger .045 difference from the highest validation accuracy to the lowest. The best batch size from the set proved to be 512 with about .68 validation accuracy.


## Question 3

I used the Karas-Tuner to optimize several of my hyperparamters. I was able to optimize the learning rate and optimizer with Karas-Tuner and I optimized the batch size by manually changing it and rerunning the model. The best hyperparameters are as follows:
- Learning Rate: .001
- Optimizer: Adam
- Batch Size: 512
- Epochs: 41

As you can see, the model trained to a .9901 accuracy on the training data. After 41 epochs, the model began to lose validation accuracy.

The best scores I was able to achieve on this model were:

25 epochs:

- Test loss: 1.060050368309021
- Test accuracy: 0.6682000160217285

41 epochs
- Test loss: 1.077089786529541
- Test accuracy: 0.6805999875068665     

![Accuracy vs Epoch](accuracy_vs_epoch.png "Accuracy vs Epoch")


## Question 4

### Part A
After running the equivalent feed forward network with the Adam optimizer and a .001 learning for 25 epochs, I got the following results:

Test loss: 1.738857626914978
Test accuracy: 0.36800000071525574

As you can see, testing the trained network led to a 36.8% validation accuracy. This score is far worse than the equivalent convolutional network's accuracy of 66.8%.


--- 


### Part B
There are 31,604 parameters in the normal feed forward network compared to the 697,046 parameters in the convolutional LeNet model. Is the greater number of parameters worth it? The convolutional model has 22.06 times more parameters than feed forward model and achieves a 84.24% increase in model accuracy. I think this increase in parameters is worth the increase in validation accuracy; especially since the training time still didn't take that long.

<br><br>

# Feed Forward LeNet Model

In [ ]:
# Sequential Model
feed_forward_model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=(32, 32, 3)),
    keras.layers.Dense(6, activation='relu'),
    keras.layers.Dense(16, activation='relu'),
    keras.layers.Dense(120, activation='relu'),
    keras.layers.Dense(84, activation='relu'),
    keras.layers.Dense(10, activation='softmax')
])

feed_forward_model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy'])

feed_forward_model.summary()


# Define the Keras TensorBoard callback.
logdir="logs/fit/" + datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = keras.callbacks.TensorBoard(log_dir=logdir)

# Train the model.
feed_forward_model.fit(
    x_train,
    train_labels, 
    batch_size=512,
    epochs=25,
    callbacks=[tensorboard_callback])

# Evaluate
score = feed_forward_model.evaluate(x_test, test_labels)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

In [3]:
X = np.array([[7,5,0,0,3,2], [6,4,5,1,4,8], [9,0,2,2,5,4], [6,3,4,7,9,8], [5,7,5,6,9,0], [7,9,0,8,2,3]])

print(X)

[[7 5 0 0 3 2]
 [6 4 5 1 4 8]
 [9 0 2 2 5 4]
 [6 3 4 7 9 8]
 [5 7 5 6 9 0]
 [7 9 0 8 2 3]]


In [9]:
le_net_model = tf.keras.models.Sequential(name='CNN_LeNet')

le_net_model.add(Layer.Conv2D(1, (3, 3), padding='same', activation='relu', input_shape=(1,6,6,1)))
# le_net_model.add(Layer.MaxPooling2D(pool_size=(2, 2), strides=(2,2)))


le_net_model.add(Layer.Flatten())
le_net_model.add(Layer.Dense(84))
le_net_model.add(Layer.Activation('relu'))

le_net_model.add(Layer.Dense(10))
le_net_model.add(Layer.Activation('softmax'))

le_net_model.compile(loss='sparse_categorical_crossentropy', optimizer=tf.keras.optimizers.Adam(learning_rate=.001), metrics=['accuracy'])

le_net_model.build()

le_net_model.summary()


Model: "CNN_LeNet"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_4 (Conv2D)            (None, 1, 6, 6, 1)        10        
_________________________________________________________________
flatten_4 (Flatten)          (None, 36)                0         
_________________________________________________________________
dense_8 (Dense)              (None, 84)                3108      
_________________________________________________________________
activation_8 (Activation)    (None, 84)                0         
_________________________________________________________________
dense_9 (Dense)              (None, 10)                850       
_________________________________________________________________
activation_9 (Activation)    (None, 10)                0         
Total params: 3,968
Trainable params: 3,968
Non-trainable params: 0
_______________________________________________________

In [ ]:









# Train the model.
le_net_model.fit(
    x_train,
    train_labels, 
    batch_size=512,
    epochs=25,
    callbacks=[tensorboard_callback, early_stopping_callback])

# Evaluate
score = le_net_model.evaluate(x_test, test_labels)
print('Test loss:', score[0])
print('Test accuracy:', score[1])